In [2]:
import os

In [3]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Pandas app\\End-to-end-ML-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
import inspect
print(inspect.getmembers(DataIngestionConfig,inspect.isfunction))

[('__delattr__', <function DataIngestionConfig.__delattr__ at 0x00000196CF340180>), ('__eq__', <function DataIngestionConfig.__eq__ at 0x00000196CF340040>), ('__hash__', <function DataIngestionConfig.__hash__ at 0x00000196CF340220>), ('__init__', <function DataIngestionConfig.__init__ at 0x00000196CF316B60>), ('__repr__', <function DataIngestionConfig.__repr__ at 0x00000196CF22F560>), ('__setattr__', <function DataIngestionConfig.__setattr__ at 0x00000196CF3400E0>)]


In [18]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [47]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2024-09-05 08:34:59,121 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-05 08:34:59,121 : INFO : common : yaml file: config\config.yaml loaded successfully
2024-09-05 08:34:59,122 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-05 08:34:59,122 : INFO : common : yaml file: params.yaml loaded successfully
2024-09-05 08:34:59,126 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-05 08:34:59,126 : INFO : common : yaml file: schema.yaml loaded successfully
2024-09-05 08:34:59,128 : INFO : common : created directory at: artifacts
2024-09-05 08:34:59,128 : INFO : common : created directory at: artifacts
2024-09-05 08:34:59,129 : INFO : common : created directory at: artifacts/data_ingestion
2024-09-05 08:34:59,129 : INFO : common : created directory at: artifacts/data_ingestion
2024-09-05 08:35:00,975 : INFO : 3447005916 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2130